In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json # for saving generated data

In [2]:
# Read file

#from google.colab import drive
#drive.mount('/content/drive/',force_remount=True)

drivePath = "drive/MyDrive/IYTE/CENG461/corncob_lowercase.txt"
localPath = "corncob_lowercase.txt"

getFromLocal = True

txtFilePath = localPath if getFromLocal else drivePath

In [3]:
# creates cache for previously generated probabilities
probTables = {}

In [4]:
# add words to array
words = []
with open(txtFilePath, "r") as f:
  for line in f.readlines():
    words.append(line.replace('\n', '') + '*')


In [5]:
ms = "a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,*"
msStates = ms.split(',')
markovChainStates = ms.split(',')

# {
#   a: {
#     0: 0,
#     total: 0,
#     a: 0,
#   ...
# probdata holds data for every state, their occurrences in first letter and occuring after specific letter
probData = {}

for state in markovChainStates:
  probData[state] = {
      '0': 0,
      'total': 0
  }


for word in words:
  for i in range(len(word)):
    letter = word[i]
    # increment total number of word
    probData[letter]['total'] = probData[letter]['total'] + 1
    if (i == 0):
      # save occurrence of letter in first position
      probData[letter]['0'] = probData[letter]['0'] + 1
      continue
    # save occurrence of letter after previous letter
    prevLetter = word[i - 1]

    probData[letter][prevLetter] = probData[letter].get(prevLetter, 0) + 1

probDF = pd.DataFrame.from_dict(probData).fillna(0.0)
probDF.sort_index(inplace=True)

In [6]:
def calculateAverageLength(words):
  total = 0

  for word in words:
    total += len(word) - 1 # minus 1 because of end of word character

  return total / len(words)

In [7]:
def calcPriorProb1(probTable, words, N):

  stateData = {}

  # initialize state data

  for state in msStates:
    stateData[state] = {
        'atloc': 0,
    }
  
  # get letter at position N
  for word in words:
    if (len(word) < N + 1):
      continue
    
    letterAtLoc = word[N]
    stateData[letterAtLoc]['atloc'] = stateData[letterAtLoc]['atloc'] + 1
  
  # calculate probabilitiy
  result = {}
  for state in msStates:
    result[state] = stateData[state]['atloc'] / probDF[state]['total']
  
  return result


In [8]:
# rescursive function to calculate joint conditional
def calculateConditional(probTable, word, state = '0'):
  
  if (len(word) == 1):
    return probTable[word][state]
  
  else:
    letter = word[-1]
    prevLetter = word[-2]

    val = calculateConditional(probTable, word[:-1])
    return probTable[letter][prevLetter] * val


In [9]:
def calcPriorProb2(probTable,N):

  stateData = {}

  # assuming word consists of same letters all, ex for state a and N = 5, our word will be "aaaaa"
  for state in msStates:
    if (state == '*'): continue # do not calculate for end of word
    word = state * N
    result = calculateConditional(probTable, word, state=state)
    stateData[state] = result

  return stateData


In [10]:
# calculate word probability using calculate conditional
def calcWordProb(probTable, word):
  return calculateConditional(probTable, word)

In [11]:
# generates prob table and saves it for order 
def generateProbTableForOrder(order):

  # check if prob table exists
  # if prob table exists use it
  # otherwise create prob table for this order

  table = probTables.get(order, 0)
  #table = 0
  #use existing prob table
  if (table != 0):
    return pd.DataFrame.from_dict(table)

  # generate all possible patterns for this order
  patterns = []

  def generatePatterns(n, letters, combination=""):
      if len(combination) == n:
          patterns.append(combination)
          return
      # Generate all possible combinations
      for letter in letters:
          generatePatterns(n, letters, combination + letter)

  generatePatterns(order, msStates[:-1])

  if (order == 1): patterns.append('0')

  # generate prob table for this order
  kOrderProbData = {}

  # initialize dictionary with msStates
  for state in msStates + ['total']:
    kOrderProbData[state] = {}

  # row data to store total occurence of pattern


  # calculate occurence of each letter after specific pattern
  for word in words:
    for i in range(len(word)):
      letter = word[i]
      if (order == 1 and i == 0):
        kOrderProbData[letter]['0'] = kOrderProbData[letter].get('0', 0) + 1

      if (i < order): continue
      # save occurrence of letter after specific patter 

      stateStr = word[i - order : i]


      kOrderProbData[letter][stateStr] = kOrderProbData[letter].get(stateStr, 0) + 1
      kOrderProbData['total'][stateStr] = kOrderProbData['total'].get(stateStr, 0) + 1

  if (order == 1):
    kOrderProbData['total']['0'] = len(words)

  df = pd.DataFrame.from_dict(kOrderProbData)

  df = df.set_index(df['total'].index)

  result = df.divide(df['total'], axis=0).fillna(0)
  result.drop(['total'], axis = 1, inplace=True)

  probTables[order] = result.to_dict()

  return result
    

In [12]:
# returns random letter from the given table and using previous pattern of order
def getKLetter(pTable, lookup, states):
  try:
    row = pTable.loc[lookup]
    arr = row.to_numpy().tolist()
    return np.random.choice(states, 1, p=arr)[0]
  except Exception as e: # if pattern not found just return random (this means probability was 0)
    return np.random.choice(states, 1)[0]



def generateWordWithOrder(order, pTable):

  word = ''


  # generate word until it ends with '*'
  while True:
    # select first letter using first order table
    if len(word) == 0:
      kProb = generateProbTableForOrder(1)
      letter = getKLetter(kProb, '0', msStates)
      word += letter
      continue
    # select second letter using first order table
    if len(word) == 1:
      kProb = generateProbTableForOrder(1)
      letter = getKLetter(kProb, word[0], msStates)
      if (letter == '*'): continue # if letter is '*' then continue 
      word += letter
      continue
    
    # select letter using smaller order table
    if (len(word) < order):
      kProb = generateProbTableForOrder(len(word))
      letter = getKLetter(kProb, word, msStates)
      word += letter
      if (letter == '*'): break
      continue
    # select letter using order table
    else:
      kProb = generateProbTableForOrder(order)
      letter = getKLetter(kProb, word[len(word) - order :len(word)], msStates)
      word += letter
      if (letter == '*'): break


  return word


In [13]:
def generateWordsWithOrder(order, M):
  # get initial prob table for order. If not exists then create it

  pTable = generateProbTableForOrder(order)

  kOrderWords = []

  for i in range(M):
    kOrderWord = generateWordWithOrder(order, pTable)
    kOrderWords.append(kOrderWord)

  return kOrderWords

In [14]:
def printWords(generatedWords, title = ""):
  print("Generated Words | " + title )
  for i in range(len(generatedWords)):
    print(f" {i + 1}. {generatedWords[i]}")


### Estimate P(L0) and P(LN | LN-1) and print it.

In [15]:
pl0 = generateProbTableForOrder(1)

print("Generated P(L0) and P(Ln | Ln-1) table")

display(pl0)

Generated P(L0) and P(Ln | Ln-1) table


,a,b,c,d,e,f,g,h,i,j,...,r,s,t,u,v,w,x,y,z,*
0,0.059869,0.055240,0.094545,0.064980,0.044536,0.044003,0.031595,0.034865,0.045999,0.008140,...,0.062537,0.114782,0.049578,0.033058,0.013956,0.026536,0.000241,0.002478,0.001480,0.000000
a,0.000382,0.050177,0.059100,0.040082,0.004147,0.009222,0.030587,0.002210,0.033779,0.000628,...,0.111951,0.060546,0.176644,0.017790,0.016426,0.009795,0.003847,0.014488,0.004420,0.019018
v,0.143665,0.000000,0.000000,0.000000,0.552242,0.000000,0.000000,0.000000,0.207797,0.000000,...,0.003119,0.000585,0.000000,0.008967,0.001754,0.000000,0.000000,0.005263,0.000000,0.001559
b,0.121587,0.032829,0.002432,0.003869,0.131425,0.001326,0.000442,0.000995,0.115950,0.004090,...,0.088096,0.039792,0.008290,0.076821,0.001990,0.000774,0.000000,0.011385,0.000111,0.013817
i,0.036663,0.015317,0.083554,0.030119,0.059651,0.019868,0.028313,0.000422,0.000211,0.000399,...,0.027515,0.117919,0.078980,0.002791,0.032300,0.000211,0.002135,0.000000,0.002956,0.003378
c,0.137167,0.000000,0.014967,0.000251,0.107333,0.000000,0.000000,0.111602,0.074586,0.000000,...,0.062481,0.011050,0.087745,0.050025,0.000000,0.000000,0.000000,0.014164,0.000251,0.046007
n,0.061866,0.003626,0.057747,0.069407,0.104878,0.015343,0.198126,0.004032,0.063490,0.002465,...,0.004206,0.089419,0.125326,0.011747,0.010093,0.002639,0.000493,0.004960,0.000899,0.093799
e,0.046462,0.006053,0.036962,0.127832,0.021427,0.013196,0.010999,0.003464,0.008107,0.000982,...,0.172134,0.152438,0.038784,0.004375,0.012089,0.007178,0.016178,0.004643,0.000964,0.115101
r,0.133063,0.009600,0.018745,0.024482,0.195234,0.005908,0.012326,0.003579,0.125167,0.000398,...,0.023289,0.083587,0.040132,0.028146,0.007697,0.003039,0.000114,0.023375,0.000170,0.097787
y,0.016768,0.004791,0.017248,0.010420,0.033777,0.002515,0.005150,0.002755,0.030902,0.000240,...,0.012217,0.046592,0.011019,0.001437,0.000000,0.004791,0.001198,0.000000,0.000000,0.696970


### Calculate the average length of a word using the given list of words and print it.

In [16]:
average = calculateAverageLength(words)

print(f"Average length of a word: {average}")

Average length of a word: 8.339098261917053


### Implement a function (calcPriorProb1) which takes the given list of words andN as input and returns P(LN). Plot the distributions for N=1,2,3,4,5 using bar plots.

In [ ]:

# plot for N = 1,2,3,4,5 prior prob 1
for i in range(1,6):
  priorProb = calcPriorProb1(probTables[1], words, i)

  fig = plt.figure()
  plt.bar(msStates, priorProb.values())
  #plt.xlabel(f"N = {i}")
  plt.ylabel("Prior Probabilities 1")
  plt.title(f"Probability Distrubiton for N = {i}")

  plt.show()


### Implement a function (calcPriorProb2) which takes P(L0),P(LN | LN-1) (estimated at Step 1) and N as input and returns P(LN). Plot the distributions for N=1,2,3,4,5 using bar plots.

In [ ]:

# plot prior prob 2
for i in range(5):
  priorProb = calcPriorProb2(probTables[1], i + 1)

  fig = plt.figure()
  plt.bar(msStates[:-1], priorProb.values())
  plt.ylabel("Prior Probabilities 2")
  plt.title(f"Probability Distrubiton for N = {i + 1}")

  plt.show()


### Calculate and print the probabilities for the following words: 

*   sad*
*   exchange*
*   antidisestablishmentarianism*
*   qwerty*
*   zzzz*
*   ae*

In [17]:
wordsToCalculateProb = ["sad*", "exchange*", "antidisestablishmentarianism*", "qwerty*", "zzzz*", "ae*"]

for word in wordsToCalculateProb:
  probRes = calcWordProb(probTables[1], word)

  print(f"Probability of word '{word}': {probRes}")

Probability of word 'sad*': 5.050673464418953e-05
Probability of word 'exchange*': 3.4260352437487854e-10
Probability of word 'antidisestablishmentarianism*': 1.6146855808797933e-31
Probability of word 'qwerty*': 0.0
Probability of word 'zzzz*': 2.250582734156605e-07
Probability of word 'ae*': 2.857933843042192e-05


#### Results
*   sad* = 5.050673464418953e-05
*   exchange* = 3.4260352437487854e-10
*   antidisestablishmentarianism* = 1.6146855808797933e-31
*   qwerty* = 0.0
*   zzzz* = 2.250582734156605e-07
*   ae* = 2.857933843042192e-05


### Generate 10 words

Sample generated words

 1. ess*
 2. mpralin*
 3. cees*
 4. mig*
 5. esoonttitss*
 6. chonaleowoourypis*
 7. sisced*
 8. litmaveisesacrr*
 9. sheel*
 10. teraly*

In [18]:
generatedWords = generateWordsWithOrder(1, 10)


printWords(generatedWords, title="First order, 10 words")

Generated Words | First order, 10 words
 1. poposeltrmpeplitidycha*
 2. gersysaliorins*
 3. rgs*
 4. wequrnapuespas*
 5. ctril*
 6. pasckeng*
 7. ceg*
 8. derolailoornstrsedic*
 9. mbated*
 10. oswagrinhy*


### By generating a synthetic dataset of size 100000, estimate the average length of a word and print it.

In [79]:
datasetSize = 100000

syntheticDataset = generateWordsWithOrder(1, datasetSize)
avg = calculateAverageLength(syntheticDataset)

print(f"Average Length of Word is '{avg}' for synthetic dataset of size '{datasetSize}'")


Average Length of Word is '9.32631' for synthetic dataset of size '100000'


### BONUS! increment the order of markov chain and genereate words

#### Order = 2

Sample generated words
 1. cers*
 2. gs*
 3. hecrushiatermirch*
 4. eddon*
 5. futting*
 6. pocuppets*
 7. fula*
 8. fech*
 9. bey*
 10. er*


In [19]:
order2Words = generateWordsWithOrder(2, 10)

printWords(order2Words, title="Second order, 10 words")

Generated Words | Second order, 10 words
 1. morefry*
 2. istemicasticifters*
 3. brablaughdiericaquen*
 4. ding*
 5. iliaing*
 6. cess*
 7. gnitylinic*
 8. stelow*
 9. pant*
 10. paged*


#### Order = 3

Sample generated words

 1. rises*
 2. ts*
 3. te*
 4. ble*
 5. dgetfullabordshipping*
 6. supplying*
 7. jectual*
 8. fully*
 9. cond*
 10. cs*

In [20]:
order3Words = generateWordsWithOrder(3, 10)

printWords(order3Words, title="Third order, 10 words")

Generated Words | Third order, 10 words
 1. pres*
 2. sm*
 3. ger*
 4. issayed*
 5. bathisery*
 6. trodidals*
 7. roids*
 8. ry*
 9. apprously*
 10. hange*


#### Order = 4

Sample generated words

1. melando*
2. fflux*
3. majoritate*
4. ces*
5. proseconvergic*
6. cellings*
7. clasper*
8. ates*
9. clamating*
10. patrian*

In [22]:
order4Words = generateWordsWithOrder(4, 10)

printWords(order4Words, title="Forth order, 10 words")

Generated Words
 1. melando*
 2. fflux*
 3. majoritate*
 4. ces*
 5. proseconvergic*
 6. cellings*
 7. clasper*
 8. ates*
 9. clamating*
 10. patrian*


#### Order = 5

Sample generated words

 1. sed*
 2. sly*
 3. ives*
 4. stairway*
 5. etting*
 6. by*
 7. assination*
 8. uckoos*
 9. ng*
 10. robes*

In [23]:
order5Words = generateWordsWithOrder(5, 10)

printWords(order5Words, title="Fifth order, 10 words")

Generated Words
 1. sed*
 2. sly*
 3. ives*
 4. stairway*
 5. etting*
 6. by*
 7. assination*
 8. uckoos*
 9. ng*
 10. robes*


#### Order = 6

* try only if you have enough ram.

Sample generated words

 1. ara*
 2. lly*
 3. wholesalers*
 4. bers*
 5. stier*
 6. umigations*
 7. sy*
 8. hers*
 9. ation*
 10. ocial*

In [25]:
order6Words = generateWordsWithOrder(6, 10)

printWords(order6Words, title="Sixth order, 10 words")

Generated Words
 1. ara*
 2. lly*
 3. wholesalers*
 4. bers*
 5. stier*
 6. umigations*
 7. sy*
 8. hers*
 9. ation*
 10. ocial*


#### Order = n


In [ ]:
n = 7

ordernWords = generateWordsWithOrder(7, 10)

printWords(ordernWords, title="Nth order, 10 words")